In [1]:
import json
import time
import datetime
import requests 
import pandas as pd
import numpy as np

# API Key IlbRy4iSNXBdEQhTyhQLXl68EVS3U2XttiNvLtpW
# 1000 Calls / Hour
# Put API Key in file. Read in API key from file instead of hardcoding in jupyter notebook. 
# http://petstore.swagger.io/
# https://api.open.fec.gov/swagger/

In [2]:
filename = 'apikey_FEC'
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

api_key = get_file_contents(filename)
print("Our API key is: %s" % (api_key))


Our API key is: IlbRy4iSNXBdEQhTyhQLXl68EVS3U2XttiNvLtpW


# Interfacing with API 
Goal: Input: Candidate, Year, District:


Output: Overall Financials, Financials in Given District 


Desired Format: .csv


May have to use: node.js, puppeter, AWS, Python Threading (? Later on ?)

### Construct Candidate_id DataFrame A

In [8]:
#Retrieve Comprehensive Candidate Dictionary

i = 1
candidate_dict = []
while True:
    candidate_id = requests.get('https://api.open.fec.gov/v1/candidates/?sort=name\
&has_raised_funds=true\
&min_first_file_date=1980\
&per_page=100\
&api_key=' + api_key + '\
&page='+str(i))
    if candidate_id.json()['results'] ==  []:
        break
    else: 
        results = candidate_id.json()['results']
        candidate_dict = candidate_dict +  results
        i += 1
print('Done')
print(i)

Done
188


In [9]:
df_general_cand = pd.DataFrame(candidate_dict)
df_general_cand.head()

,active_through,candidate_id,candidate_status,cycles,district,district_number,election_districts,election_years,federal_funds_flag,first_file_date,...,incumbent_challenge_full,last_f2_date,last_file_date,load_date,name,office,office_full,party,party_full,state
0,2018,S2UT00229,C,"[2012, 2014, 2016, 2018]",00,0.0,"[00, 00]","[2012, 2018]",False,2012-02-08,...,Open seat,2018-04-23,2018-04-23,2018-08-01T00:13:22,"AALDERS, TIMOTHY NOEL",S,Senate,CON,CONSTITUTION PARTY,UT
1,2012,H2CA01110,P,"[2012, 2014, 2016]",01,1.0,[01],[2012],False,2012-02-22,...,Challenger,2012-02-22,2012-02-22,2013-04-26T09:04:30,"AANESTAD, SAMUEL",H,House,REP,REPUBLICAN PARTY,CA
2,2018,H8CO06237,C,[2018],06,6.0,[06],[2018],False,2017-04-26,...,Challenger,2017-04-26,2017-04-26,2017-08-01T20:57:28,"AARESTAD, DAVID",H,House,DEM,DEMOCRATIC PARTY,CO
3,2008,P80002926,N,"[2006, 2008, 2010, 2012, 2014, 2016]",00,0.0,[00],[2008],False,2005-10-12,...,Open seat,2007-03-13,2007-03-13,2016-11-17T06:10:48,"AARON, LAURA DAVIS",P,President,DEM,DEMOCRATIC PARTY,US
4,2000,H0MA01024,P,"[2000, 2002, 2004]",01,1.0,[01],[2000],False,2000-02-02,...,Challenger,2000-02-02,2000-02-02,2002-04-12T00:00:00,"ABAIR, PETER JON",H,House,REP,REPUBLICAN PARTY,MA


In [10]:
df_candidate_id = df_general_cand[['name', 
                                   'candidate_id', 
                                   'incumbent_challenge_full', 
                                   'party_full',
                                   'load_date']]
number_candidates = df_candidate_id.shape[0]
print('Number of Candidates Collected: '+  str(number_candidates))
df_candidate_id.head()
#df_candidate_id.name.to_csv('candidate_names.csv')

Number of Candidates Collected: 18615


,name,candidate_id,incumbent_challenge_full,party_full,load_date
0,"AALDERS, TIMOTHY NOEL",S2UT00229,Open seat,CONSTITUTION PARTY,2018-08-01T00:13:22
1,"AANESTAD, SAMUEL",H2CA01110,Challenger,REPUBLICAN PARTY,2013-04-26T09:04:30
2,"AARESTAD, DAVID",H8CO06237,Challenger,DEMOCRATIC PARTY,2017-08-01T20:57:28
3,"AARON, LAURA DAVIS",P80002926,Open seat,DEMOCRATIC PARTY,2016-11-17T06:10:48
4,"ABAIR, PETER JON",H0MA01024,Challenger,REPUBLICAN PARTY,2002-04-12T00:00:00


### Construct Committee_id DataFrame B

In [11]:
#Retrieve Comprehensive Committee Dictionary

i = 1
committee_dict = []
while True:
    url = 'https://api.open.fec.gov/v1/committees/\
?per_page=100&sort_null_only=false&sort=name\
&sort_hide_null=false\
&page='+str(i) + '\
&min_first_file_date=2010\
&api_key=' + api_key

    committee_id = requests.get(url)
    if committee_id.json()['results'] ==  []:
        break
    else: 
        results = committee_id.json()['results']
        committee_dict = committee_dict +  results
        i += 1
print('Done')
print(i)

Done
198


In [12]:
df_general_comm = pd.DataFrame(committee_dict)
df_general_comm.head()

,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,name,organization_type,organization_type_full,party,party_full,state,treasurer_name
0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,:,None,None,W,WRITE-IN,MA,"TEIXEIRA, RASHARNDA SHELINE MISS"
1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,"10,000 LAKES VICTORY 2014",None,None,None,None,DC,ANGELIQUE HARRIS
2,[],C00563023,O,Super PAC (Independent Expenditure-Only),[2014],U,Unauthorized,T,2014-05-12,2014-05-16,2014-06-16,10 ^ 9+,None,None,None,None,CA,RANDI M WILLIS
3,[],C00687368,O,Super PAC (Independent Expenditure-Only),[2018],U,Unauthorized,Q,2018-09-12,2018-09-12,2018-09-12,11/06/18 VICTORY FUND,None,None,None,None,MS,"KEETON, LYNETTE"
4,[],C00686774,H,House,[2018],J,Joint fundraising committee,Q,2018-09-05,2018-09-05,2018-10-12,1106 FUND,None,None,None,None,DC,"MIELNIK, MEGAN"


In [33]:
df_committee_id = df_general_comm.drop(columns=['name', 'organization_type', 'organization_type_full', 'party', 'party_full', 'treasurer_name'])
empty = []
df_committee_id = df_committee_id.loc[df_committee_id.candidate_ids.map(len) > 0].reset_index()
df_committee_id.head()

,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state
0,0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,MA
1,1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,DC
2,30,"[S0MA00109, S2NV00183, S4IN00014, S4ME00055, S...",C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA
3,34,"[H6OH04082, H8OH05036]",C00528091,H,House,[2012],J,Joint fundraising committee,T,2012-08-29,2012-08-29,2012-11-29,OH
4,35,[H6NY25125],C00532879,N,PAC - Nonqualified,"[2012, 2014, 2016, 2018]",J,Joint fundraising committee,Q,2012-10-22,2012-10-22,2018-10-08,DC


In [46]:
# Add column for metric to weight the effect of contribution 
# on each candidate represented by committee

# Simple approach, to be updated at a later point

number_candidates = np.array([len(x) for x in df_committee_id.candidate_ids])**float(-1)
df_committee_id['candidate_donor_weight'] = number_candidates

In [48]:
df_committee_id.head()

,index,candidate_ids,committee_id,committee_type,committee_type_full,cycles,designation,designation_full,filing_frequency,first_file_date,last_f1_date,last_file_date,state,candidate_donor_weight
0,0,[P00008052],C00684373,P,Presidential,[2018],P,Principal campaign committee,Q,2018-08-03,2018-08-03,2018-09-09,MA,1.000
1,1,[S8MN00438],C00555748,N,PAC - Nonqualified,[2014],J,Joint fundraising committee,T,2014-01-28,2014-01-28,2014-12-18,DC,1.000
2,30,"[S0MA00109, S2NV00183, S4IN00014, S4ME00055, S...",C00495390,S,Senate,[2012],J,Joint fundraising committee,T,2011-04-15,2011-05-10,2011-10-31,VA,0.125
3,34,"[H6OH04082, H8OH05036]",C00528091,H,House,[2012],J,Joint fundraising committee,T,2012-08-29,2012-08-29,2012-11-29,OH,0.500
4,35,[H6NY25125],C00532879,N,PAC - Nonqualified,"[2012, 2014, 2016, 2018]",J,Joint fundraising committee,Q,2012-10-22,2012-10-22,2018-10-08,DC,1.000


## Retrieve Committee Donors 

In [ ]:
#    "per_page": 100,
#    "pages": 507467,
#    "count": 50746611

i = 0
last_index = 0
donor_dict = []
while True:
    url = 'https://api.open.fec.gov/v1/schedules/schedule_a/?\
per_page=100\
&last_index={}\
&sort_null_only=false\
&is_individual=true\
&min_date=2010\
&two_year_transaction_period=2018\
&sort_hide_null=false\
&api_key={}'.format(last_index, api_key) 
    
    donor = requests.get(url)
    
    
    try:
        print(donor.json())
    except json.decoder.JSONDecodeError:
        print("Encountered JSON Decoder Error")
        break
    
    if (donor.status_code == 429):
        print('429 Response Received. Sleep for an hour')
        print(datetime.datetime.now())
        print(i)
        print(last_index)
        time.sleep(3610) #Sleep for an hour & 10 Seconds before continuing.
        print('Starting up again')
        candidate_committee = requests.get(url)
        if candidate_committee.status_code == 429:
            break;
    
    print(last_index)
    print(i)
    donor = donor.json()
    donor_results = donor['results']
    
    if donor_results == []:
        break
    else: 
        donor_dict = donor_dict + donor_results 
        last_index_contribution_date = [donor['pagination']['last_indexes']['last_index'],
                                       donor['pagination']['last_indexes']['last_contribution_receipt_date']]
        last_index = last_index_contribution_date[0]
        i += 1
        
    
donor_dict

In [83]:
donor_df = pd.DataFrame(donor_dict)
columns = ['contribution_receipt_amount', 
      'contribution_receipt_date',
      'receipt_type_desc',
      'entity_type_desc',
      'contributor_city',
      'contributor_state',
      'contributor_name',
      'party_full',
      'candidate_id',
      'committee_id',
      'committee',
      'line_number_label',
      'fec_election_type_desc']
donor_df = donor_df[columns]


### Other Approach

In [30]:
# Use by_state/by_candidate, by_size/by_candidate
# QUESTION, Does this by_candidate search include donations made to committee's? 

# Use /schedules/schedule_a/by_size/by_candidate/
# Use /schedules/schedule_a/by_state/by_candidate/


# by_size/by_candidate
#  -0    $200 and under
#  -200  $200.01 - $499.99
#  -500  $500 - $999.99
#  -1000 $1000 - $1999.99
#  -2000 $2000 +

def cand_donor_size(candidate_id, cycle): 
    # Returns aggregated totals of contribution amounts that fit within defined groupings

    url_size = 'https://api.open.fec.gov/v1/schedules/schedule_a/\
by_size/by_candidate/\
?per_page=100\
&page=1\
&sort_null_only=false\
&election_full=false\
&cycle={}\
&sort_hide_null=false\
&api_key={}\
&candidate_id={}'.format(cycle, api_key, candidate_id)

    donor_cand_size = requests.get(url_size)
    print(donor_cand_size.status_code)
    donor_size_result = donor_cand_size.json()['results']
    df_dcand_size = pd.DataFrame(donor_size_result)
    return df_dcand_size

    
    
# Schedule A individual receipts aggregated by contributor state.
# This is an aggregate of only individual contributions. 
def cand_donor_state(candidate_id, cycle): 
    # Returns aggregated totals of contribution amounts that fit within defined groupings
    url_state = 'https://api.open.fec.gov/v1/schedules/schedule_a/\
by_state/by_candidate/\
?per_page=100\
&page=1\
&sort_null_only=false\
&election_full=false\
&cycle={}\
&sort_hide_null=false\
&api_key={}\
&candidate_id={}'.format(cycle, api_key, candidate_id)
    
    donor_cand_state = requests.get(url_state)
    print(donor_cand_state.status_code)
    donor_state_result = donor_cand_state.json()['results']
    df_dcand_state = pd.DataFrame(donor_state_result)
    return df_dcand_state



In [29]:
#Test
cand_donor_size('S2UT00229', 2018)
cand_donor_state('S2UT00229', 2018)

200
200


,candidate_id,cycle,state,state_full,total
0,S2UT00229,2018,OK,Oklahoma,250.00
1,S2UT00229,2018,UT,Utah,4873.23


In [28]:
# Use /committee/{committee_id}/schedules/schedule_a/by_size/
# Use /committee/{committee_id}/schedules/schedule_a/by_state/
# Extract candidate_ids supported by each committee, use DataFrame B

def comm_donor_size(committee_id):
    url = 'https://api.open.fec.gov/v1/committee/\
{}/schedules/schedule_a/\
by_size/?sort_hide_null=false\
&per_page=100\
&page=1\
&api_key={}\
&sort_null_only=false'.format(committee_id, api_key)
    
    donor_comm_size = requests.get(url)
    print(donor_comm_size.status_code)
    donor_comm_size = donor_comm_size.json()['results']
    df_dcomm_size = pd.DataFrame(donor_comm_size)
    return df_dcomm_size

def comm_donor_state(committee_id):
    url = 'https://api.open.fec.gov/v1/committee/\
{}/schedules/schedule_a/\
by_state/?sort_hide_null=false\
&per_page=100\
&page=1\
&api_key={}\
&sort_null_only=false'.format(committee_id, api_key)
    
    donor_comm_state = requests.get(url)
    print(donor_comm_state.status_code)
    donor_comm_state = donor_comm_state.json()['results']
    df_dcomm_state = pd.DataFrame(donor_comm_state)
    return df_dcomm_state



In [32]:
#Test 
comm_donor_size('C00624759')
comm_donor_state('C00624759')

200
200


,committee_id,count,cycle,idx,state,state_full,total
0,C00624759,61,2016,614768,IN,Indiana,75000.0
1,C00624759,1,2016,614767,MI,Michigan,2500.0


## Compile Comprehensive DataFrame of Donors per Candidate

### 1. For Donations made directly to a candidate


### 2. For Donations made to a committee that has a list of candidates that they support.

## Retrive Basic Candidate Information by Candidate ID

In [ ]:
#Retrieve Basic Candidate Information
def retrieve_candidate_information(candidate_id):
    url = 'https://api.open.fec.gov/v1/candidate/' + str(candidate_id) + '/?sort=name&per_page=20&page=1&api_key=' + api_key
    temp = requests.get(url)
    results = temp.json()['results']
    return pd.DataFrame(results)

#Retrieve Total/Financial Candidate Information
def retrieve_candidate_totals(candidate_id):
    url = 'https://api.open.fec.gov/v1/candidate/' + str(candidate_id) + '/totals/?api_key='+ api_key +'&sort=-cycle&per_page=20&page=1'
    temp = requests.get(url)
    results = temp.json()['results']
    return pd.DataFrame(results)
    

## Find Nancy Pelosi Data 
Full name: Nancy Patricia D'Alesandro Pelosi

In [ ]:
df_candidate_id.loc[df_candidate_id['name'].str.contains('PELOSI')]

In [ ]:
retrieve_candidate_information('H8CA05035')

In [ ]:
retrieve_candidate_totals('H8CA05035')

## Retrieve Contributions by donor

### Construct Comittee with Candidate DataFrame